<a href="https://colab.research.google.com/github/Somabss1/AIML/blob/main/cc_pdf_extractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Imports
import pandas as pd # tabular data processing
import PyPDF2 # pdf data extraction
import re # regex

Functions

In [ ]:
# Function to extract text from PDF file
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, "rb") as file:  # open pdf in read binary mode for pdf extraction
        reader = PyPDF2.PdfReader(file)  # read pdf file
        text = "".join(page.extract_text() for page in reader.pages)  # extract and join text from all pages
    return text  # return extracted text for regex pattern matching

In [ ]:
# Function to parse extracted text and convert it into a DataFrame
def parse_ccstatement_to_df(text):
    # Define the pattern for extracting data rows
    pattern = re.compile(r"(\d{4}-\d{2}-\d{2})\s+(\d{4}-\d{2}-\d{2})\s+(.*?)\s+([\d,.]+(?: CR)?)")
    matches = pattern.findall(text)

    # Create a DataFrame from the matched data
    df = pd.DataFrame(matches, columns=["Post Date", "Trans Date", "Description of Transaction", "Transaction Amount SGD"])

    return df

Load data

In [ ]:
# Local file path to pdf cc statement
pdf_path = "../data/input/sample_credit_card_statement_updated.pdf" # local path to the pdf file

# Call functions to extract pdf text and parse into df
text = extract_text_from_pdf(pdf_path)
df = parse_ccstatement_to_df(text)

print(df.shape)
df.head()

(12, 4)


,Post Date,Trans Date,Description of Transaction,Transaction Amount SGD
0,2024-08-02,2024-08-01,Grocery Store,50.00
1,2024-08-02,2024-08-02,Online Purchase,100.00
2,2024-08-04,2024-08-03,Restaurant,75.50
3,2024-08-05,2024-08-04,Credit Card Payment,200.00 CR
4,2024-08-06,2024-08-05,Taxi,20.00


In [ ]:
# Inspecting nulls and data type
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 4 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Post Date                   12 non-null     object
 1   Trans Date                  12 non-null     object
 2   Description of Transaction  12 non-null     object
 3   Transaction Amount SGD      12 non-null     object
dtypes: object(4)
memory usage: 516.0+ bytes


- Since the transaction amount is currently in string format, we need to convert it to a numerical format for analysis.
- We will also convert the date to a datetime format.
- We will also create new columns to identify a credit from debit transaction.

In [ ]:
# Create new columns for Credit and Debit
df['Credit'] = df['Transaction Amount SGD'].apply(lambda x: float(x.replace(' CR', '')) if 'CR' in x else 0)
df['Debit'] = df['Transaction Amount SGD'].apply(lambda x: float(x) if 'CR' not in x else 0)

print(df.shape)
df

(12, 6)


,Post Date,Trans Date,Description of Transaction,Transaction Amount SGD,Credit,Debit
0,2024-08-02,2024-08-01,Grocery Store,50.00,0.0,50.0
1,2024-08-02,2024-08-02,Online Purchase,100.00,0.0,100.0
2,2024-08-04,2024-08-03,Restaurant,75.50,0.0,75.5
3,2024-08-05,2024-08-04,Credit Card Payment,200.00 CR,200.0,0.0
4,2024-08-06,2024-08-05,Taxi,20.00,0.0,20.0
5,2024-08-07,2024-08-06,Utility Bill,120.00,0.0,120.0
6,2024-08-08,2024-08-07,ATM Withdrawal,200.00,0.0,200.0
7,2024-08-09,2024-08-08,Salary Credit,3000.00 CR,3000.0,0.0
8,2024-08-10,2024-08-09,Movie Tickets,30.00,0.0,30.0
9,2024-08-11,2024-08-10,Gym Membership,80.00,0.0,80.0
